In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# recfldgrn List


In this section, we want to get all available `rec@fld_grn list`.

In [2]:
# load some data reader function from recfldgrn module.
from recfldgrn.datapoint import convert_PID_to_PIDgroup
from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder
import os

Load the raw tensor (idx) from a TaskTensor Folder.

Here we treat the Task as given (the cohort, X, and Y are ready). 

Here the task name is `Task2YearXXX` (this name is meaningless, just for demonstration).

At the end, we have `recfldgrn_list` which shows all available `rec@fld_grn` we have.

In [3]:

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)
recfldgrn_list = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
recfldgrn_list

data/ProcData/TensorFolder/Task2YearXXX


['A1C@DT-DTDftGrn',
 'A1C@V-A1CNumeDftGrn',
 'Diag@DT-DTDftGrn',
 'Diag@Value-DiagDftGrn',
 'EC@BasicInfo-BasicDftGrn',
 'EC@DT_min-DTDftGrn',
 'P@age-AgeNumeDftGrn',
 'P@basicInfo-basicInfoDftGrn',
 'PN@DT-DTDftGrn',
 'PNSect@SectName-PNSctNmDftGrn',
 'PNSectSent@Sentence-Tk@TknzLLMGrn',
 'Smoking@DT-DTDftGrn',
 'Smoking@V-SmokingDftGrn']

# One recfldgrn

Here we focus on one `rec@fld_grn` to check how we parse the information.

In the python variable name, we use `recfldgrn` to refer to `rec@fld_grn`.


## Load df_Pat

In [4]:
##############
recfldgrn = 'PNSectSent@Sentence-Tk@TknzLLMGrn'
# recfldgrn = 'A1C@DT-DTDftGrn'
##############

In [5]:
# tensor_folder, get the folder of the focal recfldgrn
tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)
print(tensor_folder, '<---- tensor_folder')

# get the df_Pat: the recfldgrn grouped in the Pat (patient) level.
df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
print(df_Pat.shape, '<---- the shape of df_Pat')

# get the full_recfldgrn: it is prefix_cols + recfldgrn. 
full_recfldgrn = df_Pat.columns[0]
print(full_recfldgrn, '<---- full_recfldgrn')

# get the suffix, if it is _wgt, the recfldgrn is numeric, if _idx, the recfldgrn is categorical. 
suffix = full_recfldgrn.split('_')[-1]
assert recfldgrn in full_recfldgrn
print(suffix, '<---- suffix')
df_Pat

data/ProcData/TensorFolder/Task2YearXXX/PNSectSent@Sentence-Tk@TknzLLMGrn <---- tensor_folder
(8, 1) <---- the shape of df_Pat
P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx <---- full_recfldgrn
idx <---- suffix


,P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
PID,
P0,"[[[[[101, 2053, 2592, 2182, 1012, 102]]]], [[[..."
P1,"[[[[[101, 3178, 5987, 1037, 2694, 1012, 102]],..."
P2,"[[[[[101, 2053, 2592, 2182, 1012, 102]]]], [[[..."
P3,"[[[[[101, 2055, 2832, 2574, 2735, 1012, 102]],..."
P4,"[[[[[101, 2085, 2934, 2161, 2283, 1012, 102]],..."
P5,"[[[[[101, 2053, 2592, 2182, 1012, 102]]]], [[[..."
P6,"[[[[[101, 2053, 2592, 2182, 1012, 102]]]], [[[..."
P7,"[[[[[101, 2482, 2995, 2684, 3124, 5656, 1012, ..."


## get Batch

In [6]:
###################### take this as given
batch_PID_order = ['P0', 'P2', # 'P1', 'P3', 
                   'P4', 'P5', # 'P6', 'P7'
                  ]
######################

In [7]:
# generate a batch
df_batch = df_Pat.loc[batch_PID_order]
df_batch

,P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
PID,
P0,"[[[[[101, 2053, 2592, 2182, 1012, 102]]]], [[[..."
P2,"[[[[[101, 2053, 2592, 2182, 1012, 102]]]], [[[..."
P4,"[[[[[101, 2085, 2934, 2161, 2283, 1012, 102]],..."
P5,"[[[[[101, 2053, 2592, 2182, 1012, 102]]]], [[[..."


In [8]:
# these two functions want to convert the tensor from a nested relational_list to array.
# so we need to padding zeros to all the lists in order to make an array.
# the function `traverse` and `convert_relational_list_to_numpy` will do this for you. 

from fieldnn.utils.layerfn import traverse
from fieldnn.utils.layerfn import convert_relational_list_to_numpy

In [9]:
# get the relational_list
print(full_recfldgrn, '<--- full_recfldgrn')
relational_list = df_batch[full_recfldgrn].to_list()

P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx <--- full_recfldgrn


In [10]:
# add a batch prefix to the old full_recfldgrn
new_full_recfldgrn = 'B-' + full_recfldgrn
print(new_full_recfldgrn, '<--- new_full_recfldgrn')
suffix = '_' + full_recfldgrn.split('_')[-1]
print(suffix, '<---- suffix')

B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx <--- new_full_recfldgrn
_idx <---- suffix


In [11]:
# have a look at relational_list
relational_list

[[[[[[101, 2053, 2592, 2182, 1012, 102]]]],
  [[[[101, 2053, 2592, 2182, 1012, 102]]]],
  [[[[101, 2739, 4950, 2025, 1012, 102]],
    [[101, 4507, 3430, 2066, 2610, 2590, 2592, 3124, 1012, 102]],
    [[101, 2614, 2658, 1037, 4440, 2543, 7166, 5145, 1012, 102],
     [101, 2146, 4495, 2334, 2832, 4106, 2027, 4495, 1012, 102],
     [101, 2601, 2851, 2132, 5271, 2466, 1012, 102],
     [101, 2341, 2663, 2287, 5323, 3604, 1012, 102],
     [101, 3014, 2943, 5050, 3482, 2951, 2759, 1012, 102],
     [101, 4965, 2126, 2040, 2267, 2817, 1012, 102],
     [101, 2341, 2360, 2613, 7374, 1012, 102],
     [101, 2267, 2105, 4604, 2972, 2609, 2521, 2421, 1012, 102],
     [101, 3976, 2236, 2365, 1012, 102],
     [101, 5478, 5656, 2062, 3522, 2440, 3144, 7180, 1012, 102],
     [101, 2566, 2609, 3477, 2518, 1012, 102]],
    [[101, 2055, 6592, 3538, 3034, 2298, 2228, 2079, 1012, 102],
     [101, 2606, 3961, 2217, 2187, 3200, 2345, 2182, 1012, 102],
     [101, 2291, 3458, 2196, 2452, 2813, 2323, 1012, 102],
 

In [12]:
# have a look at df_batch
df_batch

,P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
PID,
P0,"[[[[[101, 2053, 2592, 2182, 1012, 102]]]], [[[..."
P2,"[[[[[101, 2053, 2592, 2182, 1012, 102]]]], [[[..."
P4,"[[[[[101, 2085, 2934, 2161, 2283, 1012, 102]],..."
P5,"[[[[[101, 2053, 2592, 2182, 1012, 102]]]], [[[..."


In [13]:
# padding the relatinal list to numpy


# inputs: you can check the following inputs in the above cells.
# (1): relational_list
# (2): new_full_recfldgrn
# (3): suffix


# output: this function will return a Dictionary to hold outcome
D = convert_relational_list_to_numpy(relational_list, 
                                     new_full_recfldgrn, 
                                     suffix)

# Here: k is the array name, v is the array
# B_lnP: ln means the leng. B_lnP is a scalar (zero-order). This means how many patients in the batch
# B-P_lnEC: a one-order array, each element is patient's EC number.
# every array name with ln means the length. 
# The last one is important: B-P-EC-A1C@DT-DTDftGrn_idx: the real value.
# actually, in the real practice, we will only use the last one: i.e. B-P-EC-A1C@DT-DTDftGrn_idx. 
for k, v in D.items(): print(k, v.shape)

B_lnP ()
B-P_lnEC (4,)
B-P-EC_lnPN (4, 34)
B-P-EC-PN_lnPNSect (4, 34, 1)
B-P-EC-PN-PNSect_lnPNSectSent@Sentence (4, 34, 1, 14)
B-P-EC-PN-PNSect-PNSectSent@Sentence_lnTk@TknzLLMGrn (4, 34, 1, 14, 42)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 34, 1, 14, 42, 11)


In [14]:
# have a look at B-P-EC-A1C@DT-DTDftGrn_idx, the final tensor
tensor_idx = D[new_full_recfldgrn]
print(new_full_recfldgrn, '<--- new_full_recfldgrn')
print(tensor_idx.shape, '<------- the shape of tensor_idx')

B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx <--- new_full_recfldgrn
(4, 34, 1, 14, 42, 11) <------- the shape of tensor_idx


## Validation

Here we want to do a validation to check whether the returned padding array is the same value as the origianl relational list. 

In [15]:
tensor_idx[1][9][0][0][0]

array([ 101, 2187, 2267, 2312, 3200, 2590, 2369, 1012,  102,    0,    0])

In [16]:
values_list = df_batch[full_recfldgrn].to_list()
values_list[1][9][0][0][0]

[101, 2187, 2267, 2312, 3200, 2590, 2369, 1012, 102]

In [17]:
tensor_idx[3][7][0][0][0]

array([ 101, 2053, 2592, 2182, 1012,  102,    0,    0,    0,    0,    0])

In [18]:
values_list = df_batch[full_recfldgrn].to_list()
values_list[3][7][0][0][0]

[101, 2053, 2592, 2182, 1012, 102]

# All recfldgrn

In [19]:
###################### take this as given
batch_PID_order = ['P0', 'P2', # 'P1', 'P3', 
                   'P4', 'P5', # 'P6', 'P7'
                  ]
######################

In [20]:
TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)
recfldgrn_list = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
recfldgrn_list

data/ProcData/TensorFolder/Task2YearXXX


['A1C@DT-DTDftGrn',
 'A1C@V-A1CNumeDftGrn',
 'Diag@DT-DTDftGrn',
 'Diag@Value-DiagDftGrn',
 'EC@BasicInfo-BasicDftGrn',
 'EC@DT_min-DTDftGrn',
 'P@age-AgeNumeDftGrn',
 'P@basicInfo-basicInfoDftGrn',
 'PN@DT-DTDftGrn',
 'PNSect@SectName-PNSctNmDftGrn',
 'PNSectSent@Sentence-Tk@TknzLLMGrn',
 'Smoking@DT-DTDftGrn',
 'Smoking@V-SmokingDftGrn']

## Choose a RecFldGrn List

In [21]:
# take this as given.
# we want to test whether these recfldgrn are important. 

recfldgrn_list = [
                  'P@age-AgeNumeDftGrn',
                  'P@basicInfo-basicInfoDftGrn',
    
                  'EC@BasicInfo-BasicDftGrn',
                  'EC@DT_min-DTDftGrn',
    
                  'A1C@DT-DTDftGrn',
                  'A1C@V-A1CNumeDftGrn',
                  
                  'Diag@DT-DTDftGrn',
                  'Diag@Value-DiagDftGrn',
                  
                  'PNSectSent@Sentence-Tk@TknzLLMGrn']

In [22]:
# here batch_rfg is dict to hold the padding array for each recfldgrn
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    # here we only focus on the last one: new_full_recfldgrn
    batch_rfg[new_full_recfldgrn] = tensor_idx

In [23]:
for k, v in batch_rfg.items(): print(k, v.shape)

B-P@age-AgeNumeDftGrn_wgt (4, 19)
B-P@basicInfo-basicInfoDftGrn_idx (4, 2)
B-P-EC@BasicInfo-BasicDftGrn_idx (4, 34, 2)
B-P-EC@DT_min-DTDftGrn_idx (4, 34, 7)
B-P-EC-A1C@DT-DTDftGrn_idx (4, 34, 1, 7)
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 34, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 34, 17, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 34, 17, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 34, 1, 14, 42, 11)


# TODO: Develop Dataset and DataLoader

We need to define a PyTorch Dataset and DataLoader to generate the batch automatically, based on the information we already have. This function/class will be developed based on the functions described in the following part. 
